In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydataset
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from scipy.stats import norm
import statistics
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from pandas import Series, DataFrame
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

np.random.seed(123)

In [3]:
#1.
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [12]:
#a.
df['price_per_person'] = df.total_bill/df.party_size

In [77]:
df.head()

,total_bill,tip,sex,smoker,day,time,party_size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


In [78]:
#b. I believe tyhe most important features will be total_bill and time

In [79]:
#c. party_size, and price per person
X_train, y_train = df[['total_bill', 'party_size', 'price_per_person']], df.tip

In [81]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [82]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

array([6.69247065e-34, 4.30054333e-16, 2.50210155e-08])

In [83]:
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)

In [87]:
kbest_results

,p,f
total_bill,6.692471e-34,203.357723
party_size,4.300543e-16,76.175426
price_per_person,2.502102e-08,33.213257


In [38]:
#d.
model = LinearRegression()

In [39]:
# make thing
rfe = RFE(model, n_features_to_select=2)
# fit thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [40]:
rfe.ranking_

array([1, 2, 1])

In [41]:
pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

,rfe_ranking
total_bill,1
party_size,2
price_per_person,1


In [42]:
rfe.get_support()

array([ True, False,  True])

In [43]:
X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

In [44]:
X_train_transformed.head()

,total_bill,price_per_person
0,16.99,8.495000
1,10.34,3.446667
2,21.01,7.003333
3,23.68,11.840000
4,24.59,6.147500


In [45]:
#e. the reason why the K best and recursive fit would give different results is that
# the feature uses different parameters for the test

In [50]:
#2.
def select_kbest(df):
    # make the thing
    kbest = SelectKBest(f_regression, k=2)
    # fit the thing
    _ = kbest.fit(X_train, y_train)
    kbest.scores_
    #p value: 
    value = kbest.pvalues_.max()
    return value

In [51]:
select_kbest(df)

2.5021015545972028e-08

In [52]:
#3.
def rfe(df):
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=2)
    # fit thing
    rfe.fit(X_train, y_train)
    rfe.ranking_
    pd.DataFrame(
    {
    'rfe_ranking': rfe.ranking_
    },index = X_train.columns)
    rfe.get_support()
    X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index = X_train.index,
    columns = X_train.columns[rfe.support_])
    return X_train_transformed.head()

In [53]:
rfe(df)

,total_bill,price_per_person
0,16.99,8.495000
1,10.34,3.446667
2,21.01,7.003333
3,23.68,11.840000
4,24.59,6.147500


In [56]:
#4. SWISS RFE
from pydataset import data

In [63]:
swissdf = data('swiss')
swissdf.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [64]:
X_train, y_train = swissdf[['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']], swissdf.Fertility

In [72]:
rfe = RFE(model, n_features_to_select=3)
# fit thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=3)

In [66]:
rfe.ranking_

array([2, 1, 1, 3, 1])

In [67]:
pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

,rfe_ranking
Agriculture,2
Examination,1
Education,1
Catholic,3
Infant.Mortality,1


In [68]:
rfe.get_support()

array([False,  True,  True, False,  True])

In [69]:
X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

In [70]:
X_train_transformed.head()

,Examination,Education,Infant.Mortality
Courtelary,15.0,12.0,22.2
Delemont,6.0,9.0,22.2
Franches-Mnt,5.0,5.0,20.2
Moutier,12.0,7.0,20.3
Neuveville,17.0,15.0,20.6


In [73]:
#SWISS KBEST
kbest = SelectKBest(f_regression, k=3)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [74]:
kbest.scores_
#p value: 
kbest.pvalues_

array([1.49172006e-02, 9.45043734e-07, 3.65861697e-07, 1.02852319e-03,
       3.58523805e-03])

In [75]:
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)

In [76]:
kbest_results

,p,f
Agriculture,1.491720e-02,6.408884
Examination,9.450437e-07,32.208745
Education,3.658617e-07,35.445582
Catholic,1.028523e-03,12.325096
Infant.Mortality,3.585238e-03,9.447708
